In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np

In [ ]:
images = np.load('/storage/codex/preprocessed_data/pembro_TLS_panel/210702_PembroRT_Cas19_TLSpanel_reg2/training_cells_images.npy')
annots = np.load('/storage/codex/preprocessed_data/pembro_TLS_panel/210702_PembroRT_Cas19_TLSpanel_reg2/training_cells_annots.npy')
channels = np.load('/storage/codex/preprocessed_data/pembro_TLS_panel/210702_PembroRT_Cas19_TLSpanel_reg2/training_cells_channels.npy')

channels = {k:i for i,k in enumerate(channels)}

print(images.shape, images.dtype)
print(annots.shape)
print(np.unique(annots, return_counts=True))

print(channels)

In [ ]:
# chs = ['DAPI', 'CD8', 'CD4', 'PanCytoK', 'CD138', 'CD20', 'CD45', 'CD68', 'CD31', 'CXCL13']
chs = list(channels.keys())

fig = plt.figure(figsize=(20,12))
# inds = np.random.choice(images.shape[0], 4, replace=False)

u_cts = np.unique(annots)

m = 1
for i,ct in enumerate(u_cts):
    inds = np.nonzero(annots==ct)[0] 
    ind = np.random.choice(inds)
    
    for j,ch in enumerate(chs):
        ax = fig.add_subplot(len(u_cts),len(chs),m)
        m += 1
        
        k = channels[ch]
        img = images[ind, :, :, k] / 255.
        
        b = ax.matshow(img, vmin=0, vmax=1)
        # plt.colorbar(b, ax=ax)
        ax.axis('off')
    
        if i==0:
            ax.set_title(ch)
        
        if j==0:
            ax.annotate(ct, (0.05, 0.8), xycoords='axes fraction', color='w')
